In [1]:
import win32com.client
import sqlalchemy
import time
import os
import pandas as pd
import pyodbc as py
import logging
import warnings
warnings.filterwarnings('ignore')

In [2]:
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=UG-BLR-L262;'
    r'database =AdventureWorksDW2008R2;'
    r'username=ETLdemo;'
    r'password=Devraj@@1;'
    r'Trusted_Connection=yes;'
)

In [3]:
file=open(r'demo.txt')
list1=[]
for i in (file.readlines() [-8:]):
    list1.append(i)

str1=""
for i in list1:
    str1 += i
    
body = r'''
<html>
<head>
<img src="C:\Users\prakash.raj\Downloads\header.jpg" width="100%" height="70">
<div class="dev">
<img src="C:\Users\prakash.raj\Pictures\picture3.jpg" width="30%" height="30">
<img src="C:\Users\prakash.raj\Pictures\picture2.jpg" width="30%" height="30">
<img src="C:\Users\prakash.raj\Pictures\picture1.jpg" width="30%" height="30"></div>
<title></title></head>
<body>
<h3>Email contains log files for AB testing<\h3>
</body>
</html>''' + str1

In [4]:
def sendmail():
    from_addr = 'prakash.raj@ugamsolutions.com'
    to_addr = 'prakash.raj@ugamsolutions.com'
    log_file_path = 'C:\\Users\\prakash.raj\\AB testing\\demo.txt'
    outlook = win32com.client.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to_addr
    mail.Subject = 'AB testing log'
    mail.HTMLBody = body
    mail.Attachments.Add('C:\\Users\\prakash.raj\\AB testing\\demo.txt')
    mail.Send()

In [5]:
logging.basicConfig(filename='demo.txt' ,filemode ='w' ,format='%(asctime)s %(levelname)s-%(message)s')

In [6]:
 def extract():
        try:
            #file1=open(r'demo.txt')
            #file1.truncate(0)
            cnxn = py.connect(conn_str)
            logging.warning("Connection created")
            #query being executed
            sql = """Select CustomerKey,FirstName,LastName,BirthDate,Phone from AdventureWorksDW2008R2.dbo.DimCustomer"""
            results = pd.read_sql(sql, cnxn)
            logging.warning('Query Extracted sucessfully')
            #reading excel file
            df=pd.read_excel("C:/Users/prakash.raj/Downloads/sampledatafoodsales/sampledatafoodsales.xlsx",sheet_name="FoodSales")
            logging.warning('excel extracted sucessfully')
            #reading json file
            df1=pd.read_json("C:/Users/prakash.raj/Downloads/sampledatafoodsales/food.json")
            logging.warning('json extracted sucessfully')
            #transforamtion in process
            df.set_index("CustomerKey", inplace=True)
            df1.set_index("Category", inplace=True)
            df2=results.join(df, on="CustomerKey", how="inner")
            df3=df2.join(df1, on="Category", how="inner")
            df3.set_index("order_id", inplace=True)
            load(df3)
        except exception as e:
            logging.error("data transformation block" +str(e))
        finally:
            cnxn.close()
            

In [7]:
def load(df):
    try:
        tbl='sales_detail'
        logging.warning("calling load")
        server = 'UG-BLR-L262'
        database = 'ETLtest'
        #creating engine
        sqlcon = sqlalchemy.create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
        sqlcon.connect()
        logging.warning("engine created")
        df.to_sql(tbl,schema='dbo',con=sqlcon, if_exists ='replace')
        logging.warning("table loaded sucessfully")
        #loading to dashboard
        df.to_excel("C:\\Users\\prakash.raj\\Documents\\Dashboard.xlsx", sheet_name='raw')
        logging.warning("Excel loaded sucessfully")
    except:
        logging.error("failed in loading block")     

In [8]:
try:
    open("demo.txt", "w").close()
    extract()
except NameError as e1:
    logging.exception(e1)
except SyntaxError as e2:
    logging.exception(e2)
except IndentationError as e3:
    logging.exception(e3)
except:
    logging.exception("issue in main block")
finally:
    logging.shutdown()
    sendmail()